In [62]:
import pandas as pd

pd_ds = pd.read_stata('./ZA7886_v1-0-0.dta')#.astype(str)
pd_ds = pd_ds.drop(columns=[col for col in pd_ds.columns if col not in ['isocntry', 'd17', 'sd18a', 'd72_2', 'd1', 'd11', 'd25', 'netuse', 'country']])
# Replace 'Not mentioned' with -1, "Yes" with 1, "No" with 0
# pd_ds = pd_ds.replace({'Not mentioned': -1, 'Yes': 1, 'No': 0})
#ds = datasets.Dataset.from_pandas(pd_ds)
print(f"Total number of countries: {len(pd_ds['isocntry'].unique().tolist())}")

Total number of countries: 28


In [63]:
import torch
import torch.nn as nn

#pd_ds = pd_ds[:100]

# Transform the dataset into a format that can be used by the model
pd_ds = pd_ds[~pd_ds.isin(['DK (SPONT.)']).any(axis=1)]
# Remove the categories 'Refusal/Implausible', '15 years', '98 years' from the column 'd11'
pd_ds = pd_ds[~pd_ds['d11'].isin(['Refusal/Implausible', '15 years', '98 years'])]
# Remove the category 'Refusal (SPONT.)' from the column 'd1'
pd_ds = pd_ds[~pd_ds['d1'].isin(['Refusal (SPONT.)'])]

with pd.option_context('display.max_colwidth', None, 'display.max_rows', 3, 'display.max_columns', None):
    display(pd_ds.head(5))
    for col in pd_ds.columns:
        #display(pd_ds.value_counts(col))
        # Display the string of the category and the corresponding code in a table
        display(pd_ds[col].astype('category').cat.categories)
        original_series = pd_ds[col]
        converted_series = pd_ds[col].astype('category').cat.codes
        # Display the original series and the converted series side by side
        display(pd.concat([original_series, converted_series], axis=1))

,country,isocntry,d11,d1,d25,netuse,sd18a,d72_2,d17
0,AT - Austria,AT,57,2,Large town,Everyday/Almost everyday,Fairly satisfied,Tend to agree,Fairly optimistic
...,...,...,...,...,...,...,...,...,...
5,AT - Austria,AT,19,7,Large town,Everyday/Almost everyday,Fairly satisfied,Tend to agree,Fairly optimistic


Index(['FR - France', 'BE - Belgium', 'NL - The Netherlands',
       'DE-W - Germany - West', 'IT - Italy', 'LU - Luxembourg',
       'DK - Denmark', 'IE - Ireland', 'GR - Greece', 'ES -Spain',
       'PT - Portugal', 'DE-E Germany East', 'FI - Finland', 'SE - Sweden',
       'AT - Austria', 'CY - Cyprus (Republic)', 'CZ - Czech Republic',
       'EE - Estonia', 'HU - Hungary', 'LV - Latvia', 'LT - Lithuania',
       'MT - Malta', 'PL - Poland', 'SK - Slovakia', 'SI - Slovenia',
       'BG - Bulgaria', 'RO - Romania', 'HR - Croatia'],
      dtype='object')

,country,0
0,AT - Austria,14
...,...,...
26501,SK - Slovakia,23


Index(['AT', 'BE', 'BG', 'CY', 'CZ', 'DE-E', 'DE-W', 'DK', 'EE', 'ES', 'FI',
       'FR', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL',
       'PT', 'RO', 'SE', 'SI', 'SK'],
      dtype='object')

,isocntry,0
0,AT,0
...,...,...
26501,SK,27


...

,sd18a,0
0,Fairly satisfied,1
...,...,...
26501,Not very satisfied,2


Index(['Totally agree', 'Tend to agree', 'Tend to disagree',
       'Totally disagree', 'DK (SPONT.)'],
      dtype='object')

,d72_2,0
0,Tend to agree,1
...,...,...
26501,Tend to agree,1


Index(['Very optimistic', 'Fairly optimistic', 'Fairly pessimistic',
       'Very pessimistic', 'DK (SPONT.)'],
      dtype='object')

,d17,0
0,Fairly optimistic,1
...,...,...
26501,Fairly optimistic,1


In [64]:
# Replace the string values with their corresponding codes
for col in ['isocntry', 'netuse', 'd17', 'sd18a', 'd72_2', 'd1', 'd25', 'country']:
    pd_ds[col].astype('category').cat.remove_unused_categories()
    pd_ds[col] = pd_ds[col].astype('category').cat.codes

# Convert the dataframe column 'isocntry' into a one-hot encoded tensor
#pd_ds['isocntry'] = pd_ds['isocntry'].map(lambda x: torch.nn.functional.one_hot(torch.tensor(x), num_classes=28))

# Move the column 'country' to the end of the dataframe
countries = pd_ds['country']
pd_ds = pd_ds.drop(columns=['country'])
pd_ds['country'] = countries

# Covert the age column 'd11' into an integer
pd_ds['d11'] = pd_ds['d11'].astype('category').cat.remove_unused_categories()
pd_ds['d11'] = pd_ds['d11'].astype('int')

with pd.option_context('display.max_colwidth', None, 'display.max_rows', None, 'display.max_columns', None):
    display(pd_ds.head(1))

,isocntry,d11,d1,d25,netuse,sd18a,d72_2,d17,country
0,0,57,1,2,0,1,1,1,14


In [65]:
# Perform normalization of the columns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
pd_ds['d11'] = scaler.fit_transform(pd_ds['d11'].values.reshape(-1, 1))
pd_ds['d1'] = scaler.fit_transform(pd_ds['d1'].values.reshape(-1, 1))
pd_ds['netuse'] = scaler.fit_transform(pd_ds['netuse'].values.reshape(-1, 1))
pd_ds['sd18a'] = scaler.fit_transform(pd_ds['sd18a'].values.reshape(-1, 1))
pd_ds['d72_2'] = scaler.fit_transform(pd_ds['d72_2'].values.reshape(-1, 1))
pd_ds['d17'] = scaler.fit_transform(pd_ds['d17'].values.reshape(-1, 1))
pd_ds['d25'] = scaler.fit_transform(pd_ds['d25'].values.reshape(-1, 1))
#pd_ds['country'] = scaler.fit_transform(pd_ds['country'].values.reshape(-1, 1))

with pd.option_context('display.max_colwidth', None, 'display.max_rows', None, 'display.max_columns', None):
    display(pd_ds.head(5))

,isocntry,d11,d1,d25,netuse,sd18a,d72_2,d17,country
0,0,0.506173,0.111111,1.0,0.000000,0.333333,0.333333,0.333333,14
1,0,0.160494,1.000000,1.0,0.000000,0.333333,0.333333,0.666667,14
2,0,0.259259,1.000000,1.0,0.000000,0.000000,0.000000,0.333333,14
4,0,0.580247,0.111111,1.0,0.333333,0.666667,0.000000,0.666667,14
5,0,0.037037,0.666667,1.0,0.000000,0.333333,0.333333,0.333333,14


In [66]:
# Delete the age and political party columns
pd_ds = pd_ds.drop(columns=['d11', 'd1', 'd25', 'd17', 'd72_2', 'sd18a', 'netuse'])
pd_ds['country'] = pd_ds['isocntry'] / 28
# Generate a random number between 0 and 27 for each row
#pd_ds['country'] = pd_ds['isocntry'].map(lambda x: torch.randint(0, 28, (1,)).item())
#pd_ds['isocntry'] = pd_ds['country']

In [67]:
# Create a NN with 2 hidden layers of 50 neurons each
# The model outputs a one-hot vector of size 28
# It takes as input a vector of 6 features

DROPOUT = 0.2

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Number of columns in the dataset minus the column 'isocntry'
        num_features = len(pd_ds.columns) - 1
        self.fc1 = nn.Linear(num_features, 50)
        self.fc4 = nn.Linear(50, 28)
        self.activation = nn.Tanh()
        self.softmax = nn.Softmax(dim=0)
        self.dropout = nn.Dropout(p=DROPOUT)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        #x = self.dropout(x)
        #x = self.fc3(x)
        #x = self.activation(x)
        #x = self.dropout(x)
        x = self.fc4(x)
        #x = self.activation(x)
        return x
    
net = Net()

In [68]:
# Generate train and test splits
from sklearn.model_selection import train_test_split
train, test = train_test_split(pd_ds, test_size=0.2)

In [69]:
# Train the net
import torch.optim as optim
torch.manual_seed(0)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

MINIBATCH_SIZE = 100

for epoch in range(6):  # loop over the dataset multiple times

    for i in range(0, len(train), MINIBATCH_SIZE):
        loss = 0
        # zero the parameter gradients
        optimizer.zero_grad()
        for j in range(i, min(i + MINIBATCH_SIZE, len(train))):
            # get the inputs; data is a list of [inputs, labels]
            inputs = train.iloc[j, 1:].tolist()
            inputs = torch.tensor(inputs, dtype=torch.float)
            labels = train.iloc[j, 0]
            labels = torch.tensor(labels, dtype=torch.long)

            # forward + backward + optimize
            outputs = net(inputs)
            loss += criterion(outputs, labels)
        #print(inputs, outputs, labels, loss)
        loss.backward()
        optimizer.step()

        # print statistics
        if (i / MINIBATCH_SIZE) % 15 == 0:
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, loss / MINIBATCH_SIZE))
    
    # Evaluate
    correct = 0
    total = 0
    with torch.no_grad():
        for i in range(len(test)):
            inputs = test.iloc[i, 1:].tolist()
            inputs = torch.tensor(inputs, dtype=torch.float)
            labels = test.iloc[i, 0]
            labels = torch.tensor(labels, dtype=torch.long)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 0)
            total += 1
            correct += (predicted == labels)
    
    print('Accuracy of the network on the test set: %d %%' % (
        100 * correct / total))

print('Finished Training')

[1,     1] loss: 3.379
[1,  1501] loss: 3.266
[1,  3001] loss: 3.228
[1,  4501] loss: 3.202
[1,  6001] loss: 3.129
[1,  7501] loss: 3.160
[1,  9001] loss: 3.068
[1, 10501] loss: 3.120
[1, 12001] loss: 3.049
[1, 13501] loss: 2.981
[1, 15001] loss: 2.971
[1, 16501] loss: 2.923
Accuracy of the network on the test set: 11 %
[2,     1] loss: 2.895
[2,  1501] loss: 2.879
[2,  3001] loss: 2.842
[2,  4501] loss: 2.838
[2,  6001] loss: 2.799
[2,  7501] loss: 2.828
[2,  9001] loss: 2.697
[2, 10501] loss: 2.796
[2, 12001] loss: 2.758


KeyboardInterrupt: 

In [23]:
# Test the net
dataiter = iter(pd_ds)

In [24]:
# Export the model to a torchscript model
traced_script_module = torch.jit.trace(net, inputs)
traced_script_module.save("model.pt")

In [25]:
test.head(10)

,isocntry,country
648,0,0
23859,26,26
4462,14,14
17682,3,3
23891,15,15
1927,16,16
16312,22,22
7031,11,11
14984,16,16
20681,27,27
